---

### Investigating sales data

- In this notebook, you will investigate data about sales representatives in four regions
- To answer the questions in the notebook, you need to carefully investigate the structure of the dataframe
- Use methods like `df.columns` and `df.head` to get a sense of the structure of the data before attempting the question
- Be sure you understand what the data represents. That is almost always the first step in a data analysis problem

### Sales to region

- We will get started by writing a helper function you will use throughout the problem set

In [1]:
import pandas as pd
from pandas import DataFrame
df = pd.read_csv("sales.csv")


In [2]:
df.head()

,sales_region,sales_rep_id_number,base_pay,home_city,sales
0,SW,9352,10000,Tuscon,1900
1,NE,7498,10000,Boston,1489
2,SE,2145,13000,Atlanta,4055
3,NW,8971,10000,Seattle,4624
4,NW,3434,13000,Seattle,4154


In [3]:
def get_city_to_region(input_df: DataFrame) -> dict:  
  city_dictionary={}

  ran = input_df[input_df["sales_region"] == "SW"]["home_city"]
  for i in ran:
    city_dictionary[i]="SW"
  
  
  ran = input_df[input_df["sales_region"] == "NW"]["home_city"]
  for i in ran:
    city_dictionary[i]="NW"

  ran = input_df[input_df["sales_region"] == "NE"]["home_city"]
  for i in ran:
    city_dictionary[i]="NE"
  ran = input_df[input_df["sales_region"] == "SE"]["home_city"]
  for i in ran:
    city_dictionary[i]="SE"
  
  
  return city_dictionary


  
get_city_to_region(df)
    


{'Tuscon': 'SW',
 'Pheonix': 'SW',
 'Seattle': 'NW',
 'Portland': 'NW',
 'Boston': 'NE',
 'New York': 'NE',
 'Atlanta': 'SE'}

In [4]:
# This is a 1.5 point test

df = pd.read_csv("sales.csv")
assert type(get_city_to_region(df)) == dict, "The return type should be a dictionary"

In [5]:
# This is a 1.5 point test

df = pd.read_csv("sales.csv")
assert get_city_to_region(df)["Atlanta"] == "SE", "The region for Atlanta should be SE"
assert get_city_to_region(df)["Boston"] == "NE", "The region for Boston should be NE"

In [6]:
# this is a 1 point hidden test of get_city_to_region

### BEGIN HIDDEN TESTS

df = pd.read_csv("sales.csv")
city2region = get_city_to_region(df)
df = df.dropna()
for city, region in zip(df['home_city'], df['sales_region']):
    assert city2region[city] == region

### END HIDDEN TESTS

In [7]:
def fill_in_regions(input_df: DataFrame) -> DataFrame:
    '''
    After some initial analysis, you realize that
    some data is missing for some sales reps. The sales 
    rep's city is included in the data, but the region
    is sometimes left out. 
    
    Use your get_city_to_region function to fill in any
    missing regions in the dataset. Then return the same 
    dataset with any missing regions filled in.

    Again, be sure to look carefully at the data to understand how to 
    correctly fill out this function. Examine the output of
    this function. Does it make sense?
    '''

    city2region = get_city_to_region(input_df)
    # input_df["sales_region"] = input_df["home_city"].apply(lambda x: city2region[x])

    for i, row in input_df.iterrows():
      if pd.isnull(row["sales_region"]):
        city = row["home_city"]
        input_df.loc[i, "sales_region"] = city2region[city]
    return input_df








    
 



        





    

In [8]:
# This is a 1 point test

df = pd.read_csv("sales.csv")
assert type(fill_in_regions(df)) == DataFrame, "The return type should be a data frame"

In [9]:
# This is a .5 point test

df = pd.read_csv("sales.csv")
filled = fill_in_regions(df)
assert set(filled[filled["home_city"] == "Boston"]["sales_region"]) == {"NE"}, "Sales reps from Boston should be in the NE region"

In [10]:
# This is a .5 point test

df = pd.read_csv("sales.csv")
filled = fill_in_regions(df)
assert filled["home_city"].isna().sum() == 0, "There should not be any null values in your answer"

In [11]:
def mean_sales_by_region(input_df: DataFrame) -> DataFrame:
    '''
    This is a one point function.
    Return the mean sales by region for the dataframe
    
    Use the region as the index for the output dataframe. If you pass the next
    test, you have done that
    '''
    input_df = fill_in_regions(input_df)
    input_df = input_df.drop("sales_rep_id_number", axis=1)
    # input_df.groupby("sales_region")["sales"].mean().apply(print)
    mean_sales = input_df.groupby('sales_region').mean(numeric_only=True)
    return mean_sales
mean_sales_by_region(df)

,base_pay,sales
sales_region,,
NE,11564.705882,2546.058824
NW,11709.433962,2375.845283
SE,11493.877551,2500.816327
SW,11378.723404,2590.740426


In [12]:
# This is a .5 point test
assert int(mean_sales_by_region(df).loc["NE"]["sales"]) == 2546

In [13]:
# This is a 1 point test

total = mean_sales_by_region(df).reset_index()["sales"].sum()
assert round(total) == 10013

In [14]:
# This is a 1 point hidden test

### BEGIN HIDDEN TESTS
df = pd.read_csv("sales.csv")
mini = df.head(100).copy()
total = mean_sales_by_region(mini).reset_index()["sales"].sum()
assert round(total) == 9991, round(total) 
### END HIDDEN TESTS

### Challenge problem 
- The challenge problem is hard! If you get everything but the challenge problem you will earn 8.5/10 on the assignment
- Say you have a Series of numerical values. One value will be the biggest, one will be the smallest.
- There will be some value that is at the midpoint of the series, meaning it is bigger than 1/2 of the elements of the series and smaller than half of the elements of the series.
- We can describe this value as the 50th percentile
- Similarly, if we have some value that is bigger than 10% of a series, we can call it the 10th percentile.
- To get percentiles in pandas you use the quantile function [quantile](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html) function.
- This will return the value of a series which is higher than a fraction `q` of a series
- For instance, if `q=.6` then the quantile function will return a value that is bigger than 60% of the other values in the series.

In [15]:
# here is an example of the quantile function
example = []
for n in range(1000):
    example.append({"value": n})
dfmini = pd.DataFrame(example)

dfmini['value'].quantile(q=.6) # note this value is amgiguous, 599.8 would also work

599.4

For the challenge problem, add a new column to the dataframe called `bonus` which sets a Boolean to True for all workers who sell more than 98% other sales reps **in their region**. The `bonus` column should be False otherwise.

**Important** your code must run without any pandas warnings to get full credit for the assignment. You may have to do a little research online and do a little thinking to find out how to resolve any warnings. Understanding how top to resolve and fix pandas warnings is an important programming skill. A warning will appear as a little box with an error message below your code. If you don't see a box like this, you don't have any warnings.

_Hint_: One possible warning you might see when coding will arise if you try to set values on a subset of a dataframe. Think about ways to avoid this.

In [16]:
def bonus_workers(input_df):
    '''
    Return the same data frame with an 
    additional boolean column set to True 
    for all sales reps who had sales that were
    higher than 98% of other reps in their **region**
    '''
    # YOUR CODE HERE
    df = pd.read_csv("sales.csv")
    df2 = bonus_workers(df)

    input_df = fill_in_regions(input_df)

    region_sales = input_df.groupby("sales_region")["sales"].quantile(q=0.98)

    input_df = input_df.merge(region_sales, on="sales_region", suffixes=("", "_98"))

    input_df["bonus"] = input_df["sales"] > input_df["sales_98"]

    input_df = input_df.drop("sales_98", axis=1)
    return input_df

In [17]:
# this is a 1.5 point hidden test

### BEGIN HIDDEN TESTS
import pandas as pd
df = pd.read_csv("sales.csv")
df2 = bonus_workers(df)
assert 8294 in df2[df2["bonus"]]["sales_rep_id_number"].to_list()
assert 9787 in df2[df2["bonus"]]["sales_rep_id_number"].to_list()
assert 9785 not in df2[df2["bonus"]]["sales_rep_id_number"].to_list()
### END HIDDEN TESTS

RecursionError: maximum recursion depth exceeded in comparison